# Accelerated Machine Learning

Welcome to the Accelerated Machine Learning course at AIMS 2025.

This tutorial is based on the hls4ml tutorial notebooks (https://github.com/fastmachinelearning/hls4ml-tutorial/). The dataset is described in more detail in this article (https://iopscience.iop.org/article/10.1088/1748-0221/13/07/P07027/pdf). By the end of the course you will have:

* Trained a neural network to identify different types of "jets" at the Large Hadron Collider
* Used the hls4ml python library to convert the neural network to FPGA firmware
* Examined the FPGA resource usage for the neural network
* Understood floating point vs fixed point precision and the implications for inference accuracy and resource usage
* Implemented quantization-aware training using the QKeras library
* Tested the effect of "pruning" on the inference accuray and resource usage

## Course assessment
As well as the exercises (highlighted in <span style="color: green;">green</span>), there will be a number of questions throughout the notebook to answer. The assessed questions are highlighted in <span style="color: red;">red</span>, whilst the unassessed questions are highlighted in <span style="color: orange;">orange</span>. **Both are important for your understanding**. Please upload the answers to the assessed questions via this Google Forms link: https://forms.gle/RYC28gBfhTdsqFT67.

The course assessment is out of 30. The marks for each question are provided in the notebook.

## Part 1: Understanding the jet-tagging dataset

In this section we will import the dependencies and load the jet-tagging dataset. Following this we will take some time to understand the data.

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import os
from matplotlib import pyplot as plt

%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf

tf.random.set_seed(seed)
import os
import ndjson

Jets are one of the most important objects that we reconstruct at the Large Hadron Collider experiments. 
When fundamental particles, known as quarks or gluons, are produced in the proton-proton collisions,
they undergo a process called "hadronisation".
This creates narrow sprays of particles (hadrons) which are referred to as "jets".
By reconstructing and understanding these jets, we can infer properties of the fundamental particle interactions.
The image below shows a reconstruction of a collision event which produces two back-to-back jets with extremely high energy in the CMS detector.

<img src="figures/event_display_dijet.png" alt="Dijet event display" width="400"/>

In [ ]:
# Load the dataset
data = fetch_openml('hls4ml_lhc_jets_hlf', as_frame=False, parser='liac-arff')
X, y = data['data'], data['target']
labels = np.unique(y)
features = data['feature_names']

print("Number of jets:", len(X))
print("Number of features:", len(features))
print("Different target classes:", labels)

These correspond to jets produced from a gluon (q), light-quark (q), top-quark decay (t->bqq), W-boson decay (w->qq') and a Z-boson decay (z->qq). 
Jets from the different classes will have different substructure, and we can use this information for identification.
A summary of the properties included in the dataset is provided in the table below.

<img src="figures/table_observables.png" alt="Table of observables" width="100"/>

The properties are described in this reference (https://arxiv.org/abs/1709.08705), but it is not important for you to know the exact details for this tutorial. Just that we have five different classes to predict, each of which have a different distribution in the high-dimensional jet property phase space.

Let's have a look at the 1D distributions for the different classes.

In [ ]:
colors = [
    "#E69F00",  # orange
    "#56B4E9",  # sky blue
    "#009E73",  # green
    "#CC79A7",  # pink/purple
    "#D55E00",  # red/orange
]

def plot_input_feature(ax, X, y, features, x="zlogz", classes=['g','q','t','w','z'], bins=40):
    fi = np.where(np.array(features)==x)[0][0]
    vals = X.T[fi]

    # Obtain percentiles for sensible plot
    xlo, xhi = np.percentile(vals,0.5), np.percentile(vals,99.5)

    for i, c in enumerate(classes):
        mask = (y == c)
        ax.hist(vals[mask], bins=bins, range=(xlo,xhi), label=c, color=colors[i], histtype='step')

    ax.set_ylabel("Entries")
    ax.set_xlabel(x)
    ax.legend(loc='best')

fig, axs = plt.subplots(4,4, figsize=(20,20))
for k, feature in enumerate(features):
    i = int(np.floor(k/4))
    j = k%4
    plot_input_feature(axs[i][j], X, y, features, x=feature)

plt.show()

## Part 2: Create a simple neural network for jet-tagging (offline)

We are going to use the 16-dimensional information in the dataset to classify the different kinds of jets.
Let's do this with a simple feed-forward multi-layer perceptron.

First we need to do a bit of pre-processing:

In [ ]:
# Add categorical labels and do the test-train split
le = LabelEncoder()
y = le.fit_transform(y)
y = to_categorical(y, 5)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Do standard (z-score) scaling of the input features
scaler = StandardScaler()
X_train_val = scaler.fit_transform(X_train_val)
X_test = scaler.transform(X_test)

# Save the datasets into the results folder for quick loading
np.save('results/X_train_val.npy', X_train_val)
np.save('results/X_test.npy', X_test)
np.save('results/y_train_val.npy', y_train_val)
np.save('results/y_test.npy', y_test)
np.save('results/classes.npy', le.classes_)

We will now load the relevant functionalities from tensorflow, and then build and train a neural network classifier.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1

In [ ]:
# Build the classifier
model = Sequential()
model.add(Dense(32, input_shape=(16,), name='fc1', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(Activation(activation='relu', name='relu1'))
model.add(Dense(32, name='fc2', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(Activation(activation='relu', name='relu2'))
model.add(Dense(32, name='fc3', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(Activation(activation='relu', name='relu3'))
model.add(Dense(5, name='output', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(Activation(activation='softmax', name='softmax'))

# Print summary
model.summary()

<div style="background-color:#FFE5CC">
    
### Questions (unassessed)
* Do you understand the architecture of this MLP?
* What do the `kernel_initializer` and `kernal_regularizer` arguments for the dense layers do?
* Why might an L1 regularisation be appropriate for a network with limited resources?

In [ ]:
# Train the network
N_epochs = 20
adam = Adam(learning_rate=0.0001)
model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])

In [ ]:
result = model.fit(
    X_train_val,
    y_train_val,
    batch_size=128,
    epochs=N_epochs,
    validation_split=0.25,
    shuffle=True,
)

In [ ]:
# Plot the training history
history = result.history

fig, axs = plt.subplots(1,2, figsize=(12,4))

epochs = np.linspace(1, N_epochs, N_epochs)
axs[0].plot(epochs, history['loss'], c='blue', label='Train')
axs[0].plot(epochs, history['val_loss'], c='orange', label='Val')
axs[0].set_xlabel("Epochs")
axs[0].set_ylabel("Loss")
axs[0].legend(loc='upper right')

axs[1].plot(epochs, history['accuracy'], c='blue', label='Train')
axs[1].plot(epochs, history['val_accuracy'], c='orange', label='Val')
axs[1].set_xlabel("Epochs")
axs[1].set_ylabel("Accuracy [%]")
axs[1].legend(loc='lower right')

<div style="background-color:#FFE5CC">

### Questions (unassessed)
* What conclusions can you draw from the shape of the loss curve?
    * **Hint**: comment on overtraining/undertraining

This tutorial is not about obtaining the best performance for the "software" neural network classifier, so we will not spend any more time improving the performance of this network. In any case, we should evaluate the performance on the test dataset to provide a "software" baseline.

Let's first calculate the global accuracy. This is defined as the fraction of correctly classified jets:

In [ ]:
y_keras = model.predict(X_test)
keras_accuracy = np.sum(np.argmax(y_test, axis=1) == np.argmax(y_keras, axis=1))/y_test.shape[0]

print(f"Achieved test accuracy of {(100 * keras_accuracy):.2f}%")

<div style="background-color:#C2F5DD">

### Exercise
Let's understand the performance in more detail beyond the global accuracy. It is interesting to evaluate how the model classifies one type of jet vs another. In this exercise, we will:

1) Plot the output probabilities from the neural network (multi-)classifier
2) Evaluate the ROC curves
3) Plot the "confusion matrix" which tells us the fraction of predicted labels for each "truth" jet type

In [ ]:
# Exercise 1

# Build masks for truth class
masks = {}
for truth_label in labels:
    masks[truth_label] = np.sum((labels == truth_label)*y_test, axis=1, dtype='bool')

fig, axs = plt.subplots(3,2, figsize=(12,12))
axs[2,1].axis('off')

# Loop over predicted classes and plot the probabilities from the classifier
# Store the probabilities for future use
probs = {}
for j, pred_label in enumerate(list(labels)):

    x = int(np.floor(j/2))
    y = j%2
    
    arg = np.where(labels==pred_label)[0][0]
    prob = y_keras.T[arg]
    probs[pred_label] = prob

    lo, hi = 0, 1

    # Plot probability distribution for each truth class
    for i, truth_label in enumerate(list(labels)):
        mask = masks[truth_label]
        axs[x][y].hist(prob[mask], bins=40, range=(lo,hi), label=truth_label, histtype='step', color=colors[i])

    axs[x][y].set_xlabel(f"Prob({pred_label})")
    axs[x][y].set_ylabel("Entries")
    axs[x][y].set_yscale("log")
    axs[x][y].legend(loc='lower left', title="Truth class")

In [ ]:
# Exercise 2
from sklearn.metrics import roc_curve, roc_auc_score

fig, axs = plt.subplots(3,2, figsize=(12,16))
axs[2,1].axis('off')

for j, label in enumerate(list(labels)):

    x = int(np.floor(j/2))
    y = j%2

    y_true = masks[label]
    y_pred = probs[label]
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    axs[x][y].plot(fpr, tpr, label=f"All others (AUC={auc:.2f})", color='black')
    for i in range(0, len(thresholds), 2000):
        axs[x][y].text(fpr[i], tpr[i], f"{thresholds[i]:.2f}", fontsize=6, ha='right', va='bottom', color='black')

    # Plot for each pair
    for other_label in labels:
        if other_label == label:
            continue

        mask_pair = masks[label]+masks[other_label]
        fpr, tpr, thresholds = roc_curve(y_true[mask_pair], y_pred[mask_pair])
        auc = roc_auc_score(y_true[mask_pair], y_pred[mask_pair])
        axs[x][y].plot(fpr, tpr, label=f"{other_label} (AUC={auc:.2f})", color=colors[np.where(labels==other_label)[0][0]], ls='--')

    axs[x][y].set_xlabel("False positive rate")
    axs[x][y].set_ylabel("True positive rate")
    axs[x][y].set_title(f"{label} vs X (using Prob({label}))")
    axs[x][y].legend(loc='lower right')

In [ ]:
# Exercise 3
# This tell us the fraction of each true class landing in each predicted class
from sklearn.metrics import confusion_matrix

y_true_label = np.argmax(y_test, axis=1)
y_pred_label = np.argmax(y_keras, axis=1)

cm = confusion_matrix(y_true_label, y_pred_label, normalize='true')

fig, ax = plt.subplots()
im = ax.imshow(cm, cmap='Blues')

ax.set_xticks(np.arange(len(labels)))
ax.set_yticks(np.arange(len(labels)))
ax.set_xticklabels(labels)
ax.set_yticklabels(labels)

ax.set_xlabel('Predicted class')
ax.set_ylabel('True class')
ax.set_title("Normalized by true class")

for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, f"{cm[i, j]:.2f}", ha='center', va='center', color='black')

<div style="background-color:#FFCCCC">

### Questions (assessed)

These quiz questions will test your understanding of the results we have just obtained.

* Which true jet class is most often misidentified as a gluon-jet (g)? [1 mark]
* What do the small numbers along the ROC curves represent? [1 mark]
* Which true jet class is easiest to identify? What is special about the substructure of this type of jet compared to the others? [2 marks]

Please add your answers to the Google Forms linked at the top of this notebook.

## Part 3: Online jet-tagging -- convert the model to FPGA firmware

As discussed in the lectures, we will benefit hugely from implementing Machine Learning in the CMS trigger - the part of the detector that decides which data should be written to disk. For this "online" jet tagging, we need to synthesize the neural network classifier trained in the previous section into FPGA firmware, and ensure that it fits within the latency and resource usage requirements.

In this section we will convert the neural network trained with tensorflow into FPGA firmware. This is known as High-Level Synthesis (HLS) which allows us to write software-like code (e.g. in C, C++) and automatically compile it into hardware code (logic gates, registers, interconnects) that can be synthesized on an FPGA.

We will use the `hls4ml` package with an Intel OneAPI backend to do this. We have heard about these packages in the lecture slides. The tutorial will focus on optimising the resource usage, as latency tests require a full simulation of the data flow within the FPGA.

Before starting, there's one key property of FPGA algorithms which must be highlighted.

### Datatypes in HLS
Fixed point representation is used instead of floating point. For example the `hls4ml` default `fixed<16,6>` corresponds to:
```
+-------------------------+---------------------------+
|  Integer part (6 bits)  | Fractional part (10 bits) |
+-------------------------+---------------------------+
| 101010                  |                1010101010 |
+-------------------------+---------------------------+
|           Full bitwidth (16 bits)                   |
+-------------------------+---------------------------+
```
You can read more here: https://github.com/hlslibs/ac_types/blob/v3.7/pdfdocs/ac_datatypes_ref.pdf

This has important consequences for the synthesized models. At the end of this section we will see how the performance and resource-usage depend on bit-width length used for the network weights and biases during inference. In part 4, we will see how to incorporate bit-widths into the training process to maintain performance for a very small numbers of bits.

Let's set up the converter:

In [ ]:
import hls4ml

config = hls4ml.utils.config_from_keras_model(model, granularity='model', backend='oneAPI')
print("-----------------------------------")
print("Configuration")
print(config)
print("-----------------------------------")

What do the different items in the configuration dictionary correspond to? The most important for you to understand are:

* `Precision`: specifies the numeric format to use in the generated hardware. This choice affects both resource usage and model accuracy. The precision can be specified globally (as shown above) or separate for each neural network layer.
* `ReuseFactor`: controls hardware reuse, such that `1` means no reuse i.e. the design uses as many hardware resources as needed to execute all operations in parallel. The lower this factor the faster the inference (lower latency) but more FPGA area (i.e. resources) is used. This factor can be tuned to meet the design constraints (parallelisation).
* `Strategy`: tells hls4ml to optimize for the lowest possible latency. Other strategies include `Resource`, which optimizes for minimal area.

We will now convert the keras model we trained for an Intel "Agilex7" FPGA, a family of high-end FPGAs which are designed for performance-demanding and power-efficient applications, like AI acceleration.

In [ ]:
hls_model = hls4ml.converters.convert_from_keras_model(
    model, hls_config=config, backend='oneAPI', output_dir='model_1/hls4ml_prj', part='Agilex7'
)
hls_model.compile()

Now that we have converted the model, we can evaluate the global accuracy (i.e. fraction of correctly classified jets). This should show little change from the software model.

In [ ]:
X_test = np.ascontiguousarray(X_test)
y_hls = hls_model.predict(X_test)

hls_accuracy = np.sum(np.argmax(y_test, axis=1) == np.argmax(y_hls, axis=1))/y_test.shape[0]

In [ ]:
print(f"Achieved test accuracy of {(100 * hls_accuracy):.2f}%")

### Create a report of the FPGA resource usage

Now that we have a classifier which has been converted (and compiled) for use on an FPGA, we can emulate the design build and estimate the resource usage to see if it is compatible with our resource requirements. The report is produced in the project directory. We will will print out the key information from the build report.
* **Note**: ignore the seg fault

In [ ]:
hls_model.build(build_type='report')

In [ ]:
with open("model_1/hls4ml_prj/build/myproject.report.prj/reports/resources/json/summary.ndjson", "r") as f:
    summary = ndjson.load(f)

resource_names = list(filter(lambda x: x["name"] == "Estimated Resource Usage", summary))[0]['columns'][1:-1]
available = list(filter(lambda x: x["name"] == "Available", summary))[0]['data'][:-1]
estimated_resources = list(filter(lambda x: x["name"] == "Total", summary))[0]['data'][:-1]

print("~~~~~~~~~~~~~~ Resource usage ~~~~~~~~~~~~~~")
for i, resource in enumerate(resource_names):
    print(f"--> {resource}:")
    print(f"      * Available resource: {available[i]}")
    print(f"      * Used resource (estimated): {estimated_resources[i]}")
    print(f"      * Percentage of used resource (estimated): {100*float(estimated_resources[i])/float(available[i]):.2f}%")

The different resource types correspond to:
* ALUTs: adaptive look-up tables. Implement logic expressions/functions
* FFs: flip-flops. Store one bit of data; used for variables/registers
* RAMs: On-chip block RAMs. Used for storing network weights, activations etc.
* DSPs: Digital Signal Processing blocks. Optimized for multiply/add functionalities which are ideal for NN layers.
* MLABs: Memory Logic Array blocks. Small, distributed memory with lower latency and smaller capacity than RAMs.

A simple (reductive) guide for when compiling a neural network with hls4ml. If it uses:
* Many DSPs --> You are doing lots of multiple/add operations e.g. dense layers/convolutions
* Lots of RAMs/MLABs --> You are storing many activations, weights, or intermediate values.
* High ALUT/FF usage --> Your logic is complex e.g. control flow, state machines.

**So even a quite simple neural network design (only ~3000 trainable parameters) can take up a sizeable chunk of the FPGA.**
* How can we accelerate more complex ML algorithms without being limited by the resources?
* How can we store many ML algorithms on the same FPGA?

We need to think smarter in terms of the training!

<div style="background-color:#C2F5DD">
    
### Exercise

Now you have learnt how to convert a software neural network into FPGA firmware, let's see how changing the bitwidth used for the network layer weights and biases during inference affects the accuracy and resource usage.

Your task is to loop over several bitwidths and plot the accuracy and resource usage as a function of bitwidth. 

</div>

In [ ]:
# SOLUTION: you can base your solution using the skeleton code below
# We will fix the number of integer bits at 6, use the rest for fractional component

# Loop over a set of bit widths
bit_widths = [8, 10, 12, 14, 16, 18, 20]

hls_accuracy_list = []
hls_summary_list = []

for bit_width in bit_widths:
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print(f" --> Testing bit width: {bit_width}")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    # Set default bit_width in the config
    config_tmp = config
    config_tmp['Model']['Precision']['default'] = f"fixed<{bit_width},6>"

    # Convert and compile the model
    # -- ADD CODE HERE --

    # Evaluate the accuracy of the model
    # -- ADD CODE HERE --
    hls_accuracy_list.append(accuracy)

    # Emulate the build and evaluate the resource usage
    # -- ADD CODE HERE
    with open("PATH-TO-REPORT/summary.ndjson", "r") as f:
        summary = ndjson.load(f)
    hls_summary_list.append(summary)

In [ ]:
# SOLUTION: Extract resource usage from summary jsons
resource_usage = {}
for summ in hls_summary_list:
    resource_names = list(filter(lambda x: x["name"] == "Estimated Resource Usage", summ))[0]['columns'][1:-1]
    available = list(filter(lambda x: x["name"] == "Available", summ))[0]['data'][:-1]
    estimated_resources = list(filter(lambda x: x["name"] == "Total", summ))[0]['data'][:-1]

    for i, resource in enumerate(resource_names):
        r = resource.split(" ")[0]
        if r not in resource_usage:
            resource_usage[r] = [float(estimated_resources[i])/float(available[i])]
        else:
            resource_usage[r].append(float(estimated_resources[i])/float(available[i]))

In [ ]:
# SOLUTION: Plot accuracy and resource usage as a function of the bit widths
fig, axs = plt.subplots(2,3, figsize=(16,8))

# Top left: accuracy
axs[0][0].plot(bit_widths, 100*np.array(hls_accuracy_list), marker='o')
axs[0][0].set_xlabel("Bit widths")
axs[0][0].set_ylabel("Accuracy [%]")
axs[0][0].axhline(20, color='red', ls='--', label="Random choice")
axs[0][0].set_ylim(0,100)
axs[0][0].legend(loc='best')

# Top middle: ALUT usage
axs[0][1].plot(bit_widths, 100*np.array(resource_usage['ALUTs']), marker='o')
axs[0][1].set_xlabel("Bit widths")
axs[0][1].set_ylabel("ALUTs [%]")

# Top right: FF usage
axs[0][2].plot(bit_widths, 100*np.array(resource_usage['FFs']), marker='o')
axs[0][2].set_xlabel("Bit widths")
axs[0][2].set_ylabel("FFs [%]")

# Bottom left: BRAM usage
axs[1][0].plot(bit_widths, 100*np.array(resource_usage['RAMs']), marker='o')
axs[1][0].set_xlabel("Bit widths")
axs[1][0].set_ylabel("RAMs [%]")

# Top middle: DSP usage
axs[1][1].plot(bit_widths, 100*np.array(resource_usage['DSPs']), marker='o')
axs[1][1].set_xlabel("Bit widths")
axs[1][1].set_ylabel("DSPs [%]")

# Top right: MLAB usage
axs[1][2].plot(bit_widths, 100*np.array(resource_usage['MLABs']), marker='o')
axs[1][2].set_xlabel("Bit widths")
axs[1][2].set_ylabel("MLABs [%]")

<div style="background-color:#FFCCCC">

### Questions (assessed)

* Please upload a screenshot of the graphs produced in this exercise which show the accuracy and resource-usage (LUTs, FFs, DSPs, BRAM, MLAB) as a function of the bit-widths used during inference. [3 marks]
* What conclusions can be drawn from the graphs? [2 marks]

## Part 4: Quantization-Aware Training

In the previous exercise we have seen the trade-off between accuracy and resource usage when using different bit-widths **during inference**.

As discussed in the lecture slides, we can achieve similar levels of accuracy with significantly lower bit widths by **quantizing during the network training**. We will use the QKeras library to emulate fixed-point quantization during the forward pass of the training process. This ensures that network learns to optimally classify the different jet types within the fixed bit-width budget.

By lowering the bit widths we can:
* Reduce the resource usage (lower memory footprint)
* Speed up the inference process (reduce latency)

These are essential for AI acceleration using FPGAs! Let's start by importing the `qkeras` library. We will then look at how the `quantized_bit` behaves vs the full floating point equivalent.

In [ ]:
import qkeras
from qkeras import QDense
from qkeras.quantizers import quantized_bits

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.linspace(-4,4,1000), quantized_bits(4,1,1,alpha=1)(np.linspace(-4,4,1000)))
ax.plot(np.linspace(-4,4,1000), np.linspace(-4,4,1000))

ax.legend(["Quantized", "Floating"])

<div style="background-color:#FFE5CC">

### Questions (unassessed)
* Do you understand what the plot is showing?

### Creating a quantized model

To create a "quantized" neural network we using the `QDense` layer. This requires setting the `kernel_quantizer` and `bias_quantizer`, which control how the layer's weights and biases are quantized, respectively i.e. how they are converted from full-precision floating-point numbers to lower-bit fixed-point formats during training and inference.

We will reduce the model size substantially by only using 4 bits (1 integer) for each weight and bias in the network. This means each weight and bias can only take 1 of 16 different values (as shown in the plot above). Note, the number of training parameters does not change.

In [ ]:
qmodel = Sequential()
qmodel.add(QDense(32, input_shape=(16,),
                  name='fc1',
                  kernel_quantizer = quantized_bits(4,1,1,alpha=1),
                  bias_quantizer = quantized_bits(4,1,1,alpha=1),
                  kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
qmodel.add(Activation(activation='relu', name='relu1'))
qmodel.add(QDense(32, 
                  name='fc2', 
                  kernel_quantizer = quantized_bits(4,1,1,alpha=1),
                  bias_quantizer = quantized_bits(4,1,1,alpha=1),                  
                  kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
qmodel.add(Activation(activation='relu', name='relu2'))
qmodel.add(QDense(32, 
                  name='fc3', 
                  kernel_quantizer = quantized_bits(4,1,1,alpha=1),
                  bias_quantizer = quantized_bits(4,1,1,alpha=1),                  
                  kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
qmodel.add(Activation(activation='relu', name='relu3'))
qmodel.add(QDense(5, 
                  name='output', 
                  kernel_quantizer = quantized_bits(4,1,1,alpha=1),
                  bias_quantizer = quantized_bits(4,1,1,alpha=1),                  
                  kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
qmodel.add(Activation(activation='softmax', name='softmax'))

qmodel.summary()

Now let's train the network. With qkeras the fixed-point format is only used for the forward pass, and we revert back to the floating-point format for the backwards pass (backpropagation). This ensures that the loss is smooth and differentiable, with respect to the network weights and biases.

In [ ]:
adam = Adam(learning_rate=0.0001)
qmodel.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])

# We will initialise with the weights from the full floating-points model (translated into the quantized_bit format)
qmodel.set_weights(model.get_weights())

In [ ]:
N_epochs = 20
result_qkeras = qmodel.fit(
    X_train_val,
    y_train_val,
    batch_size=128,
    epochs=N_epochs,
    validation_split=0.25,
    shuffle=True,
)

In [ ]:
# Plot the training history and compare to the normal keras training
history_qkeras = result_qkeras.history

fig, axs = plt.subplots(1,2, figsize=(12,4))

epochs = np.linspace(1, N_epochs, N_epochs)
axs[0].plot(epochs, history['loss'], c='blue', label='Train')
axs[0].plot(epochs, history['val_loss'], c='orange', label='Val')
axs[0].plot(epochs, history_qkeras['loss'], c='blue', label='Train (qkeras)', ls='--')
axs[0].plot(epochs, history_qkeras['val_loss'], c='orange', label='Val (qkeras)', ls='--')
axs[0].set_xlabel("Epochs")
axs[0].set_ylabel("Loss")
axs[0].legend(loc='upper right')

axs[1].plot(epochs, history['accuracy'], c='blue', label='Train')
axs[1].plot(epochs, history['val_accuracy'], c='orange', label='Val')
axs[1].plot(epochs, history_qkeras['accuracy'], c='blue', label='Train (qkeras)', ls='--')
axs[1].plot(epochs, history_qkeras['val_accuracy'], c='orange', label='Val (qkeras)', ls='--')
axs[1].set_xlabel("Epochs")
axs[1].set_ylabel("Accuracy [%]")
axs[1].legend(loc='lower right')

<div style="background-color:#FFE5CC">

### Questions (unassessed)
* What conclusions would you draw by comparing the loss curves?

Now we can use the hls4ml package to convert the "quantized" network into FPGA firmware via HLS. The key difference is that we are now using a more complex configuration, with a separate config per layer name. For this we need to set the `granularity` argument to `"name"`. Let's set up the configuration and look at the contents (ignoring the warning):

In [ ]:
config = hls4ml.utils.config_from_keras_model(qmodel, granularity='name', backend='oneAPI')
print("-----------------------------------")
print("Configuration")
print(config)
print("-----------------------------------")

<div style="background-color:#FFE5CC">

### Questions (unassessed)
* Do you understand what has changed in the configuration for this "quantization-aware" network?

In [ ]:
# Let's do the HLS conversion
hls_model = hls4ml.converters.convert_from_keras_model(
    qmodel, hls_config=config, backend='oneAPI', output_dir='model_2/hls4ml_prj', part='Agilex7'
)
hls_model.compile()

In [ ]:
# And evaluate the accuracy on the data set
X_test = np.ascontiguousarray(X_test)
y_hls = hls_model.predict(X_test)

hls_accuracy = np.sum(np.argmax(y_test, axis=1) == np.argmax(y_hls, axis=1))/y_test.shape[0]

In [ ]:
print(f"Achieved test accuracy of {(100 * hls_accuracy):.2f}%")

So we have managed to achieve roughly the same accuracy as the baseline model with much lower bit-widths.

Let's build the QKeras trained model and get an estimate for the resource usage.

In [ ]:
hls_model.build(build_type='report')

In [ ]:
with open("model_2/hls4ml_prj/build/myproject.report.prj/reports/resources/json/summary.ndjson", "r") as f:
    summary = ndjson.load(f)

resource_names = list(filter(lambda x: x["name"] == "Estimated Resource Usage", summary))[0]['columns'][1:-1]
available = list(filter(lambda x: x["name"] == "Available", summary))[0]['data'][:-1]
estimated_resources = list(filter(lambda x: x["name"] == "Total", summary))[0]['data'][:-1]

print("~~~~~~~~~~~~~~ Resource usage ~~~~~~~~~~~~~~")
for i, resource in enumerate(resource_names):
    print(f"--> {resource}:")
    print(f"      * Available resource: {available[i]}")
    print(f"      * Used resource (estimated): {estimated_resources[i]}")
    print(f"      * Percentage of used resource (estimated): {float(estimated_resources[i])/float(available[i]):.2f}%")

resource_usage_qkeras = {}
for i,resource in enumerate(resource_names):
    r = resource.split(" ")[0]
    resource_usage_qkeras[r] = float(estimated_resources[i])/float(available[i])

<div style="background-color:#C2F5DD">
    
### Exercise: 

How do these resource usage numbers compare to the baseline model?

1) Plot the accuracy and resource usage statistics vs bit-widths for the baseline model (previous exercise), adding an extra point for the QKeras trained network
2) Plot the ALUTs/FFs usage vs accuracy for different bin-widths (previous exercise), and add the point from the QKeras training

What conclusions can be drawn?

In [ ]:
# Exercise 1
fig, axs = plt.subplots(2,3, figsize=(16,8))

bit_width_qkeras = 4

# Top left: accuracy
axs[0][0].plot(bit_widths, 100*np.array(hls_accuracy_list), marker='o', label='Baseline')
axs[0][0].plot(bit_width_qkeras, 100*np.array(hls_accuracy), marker='*', label='QKeras')
axs[0][0].set_xlabel("Bit widths")
axs[0][0].set_ylabel("Accuracy [%]")
axs[0][0].axhline(20, color='red', ls='--', label="Random choice")
axs[0][0].set_ylim(0,100)
axs[0][0].legend(loc='best')

# Top middle: ALUT usage
axs[0][1].plot(bit_widths, 100*np.array(resource_usage['ALUTs']), marker='o', label='Baseline')
axs[0][1].plot(bit_width_qkeras, 100*resource_usage_qkeras['ALUTs'], marker='*', label='QKeras')
axs[0][1].set_xlabel("Bit widths")
axs[0][1].set_ylabel("ALUTs [%]")
axs[0][1].legend(loc='best')

# Top right: FF usage
axs[0][2].plot(bit_widths, 100*np.array(resource_usage['FFs']), marker='o', label='Baseline')
axs[0][2].plot(bit_width_qkeras, 100*resource_usage_qkeras['FFs'], marker='*', label='QKeras')
axs[0][2].set_xlabel("Bit widths")
axs[0][2].set_ylabel("FFs [%]")
axs[0][2].legend(loc='best')

# Bottom left: RAM usage
axs[1][0].plot(bit_widths, 100*np.array(resource_usage['RAMs']), marker='o', label='Baseline')
axs[1][0].plot(bit_width_qkeras, 100*resource_usage_qkeras['RAMs'], marker='*', label='QKeras')
axs[1][0].set_xlabel("Bit widths")
axs[1][0].set_ylabel("RAMs [%]")
axs[1][0].legend(loc='best')

# Top middle: DSP usage
axs[1][1].plot(bit_widths, 100*np.array(resource_usage['DSPs']), marker='o', label='Baseline')
axs[1][1].plot(bit_width_qkeras, 100*resource_usage_qkeras['DSPs'], marker='*', label='QKeras')
axs[1][1].set_xlabel("Bit widths")
axs[1][1].set_ylabel("DSPs [%]")
axs[1][1].legend(loc='best')

# Top right: MLAB usage
axs[1][2].plot(bit_widths, 100*np.array(resource_usage['MLABs']), marker='o', label='Baseline')
axs[1][2].plot(bit_width_qkeras, 100*resource_usage_qkeras['MLABs'], marker='*', label='QKeras')
axs[1][2].set_xlabel("Bit widths")
axs[1][2].set_ylabel("MLABs [%]")
axs[1][2].legend(loc='best')

In [ ]:
# Exercise 2
# Plot ALUT usage vs accuracy
fig, axs = plt.subplots(1,2, figsize=(12,4))
axs[0].plot(100*np.array(resource_usage['ALUTs']), 100*np.array(hls_accuracy_list), marker='o', label='Baseline')
axs[0].plot(100*resource_usage_qkeras['ALUTs'], 100*np.array(hls_accuracy), marker='*', label='QKeras')
axs[0].set_xlabel("ALUTs [%]")
axs[0].set_ylabel("Accuracy [%]")
axs[0].set_ylim(0,100)
axs[0].legend(loc='best')

# Plot ALUT usage vs accuracy
axs[1].plot(100*np.array(resource_usage['FFs']), 100*np.array(hls_accuracy_list), marker='o', label='Baseline')
axs[1].plot(100*resource_usage_qkeras['FFs'], 100*np.array(hls_accuracy), marker='*', label='QKeras')
axs[1].set_xlabel("FFs [%]")
axs[1].set_ylabel("Accuracy [%]")
axs[1].set_ylim(0,100)
axs[1].legend(loc='best')

This is a great achievement! We have obtained a classifier with the same performance for much smaller resource usage. These "tricks" have a huge consequence for accelerated artificial intelligence. Not only can we vastly speed-up the inference task (e.g. for use in low latency environments like the CMS level-1 trigger), but we can substantially reduce the carbon footprint involved in the inference. This will become increasingly important in an AI-dominated world.

<div style="background-color:#C2F5DD">

### Exercise

With the quantised-aware training, investigate how low we can take the bitwidth to still achieve the same accuracy as the baseline model.

Loop over several bitwidths and plot the accuracy and resource-usage as a function of bitwidth.

The first block of code, plots the fixed-point vs floating-point comparison for different bitwidths to visualise the problem. 

You will then need to loop over different bit widths and retrain the network to see what performance can be achieved. Note, the notation for bit width is: <total number of bits, number of integer bits>.

</div>

In [ ]:
fig, axs = plt.subplots(2,4, figsize=(16,6))

for i in range(0,8):
    x = 0 if i < 4 else 1
    y = i % 4

    axs[x][y].plot(np.linspace(-4,4,1000), np.linspace(-4,4,1000))
    axs[x][y].plot(np.linspace(-4,4,1000), quantized_bits(i+1,1,1,alpha=1)(np.linspace(-4,4,1000)))
    if i > 0:
        axs[x][y].plot(np.linspace(-4,4,1000), quantized_bits(i+1,2,1,alpha=1)(np.linspace(-4,4,1000)))
    
    axs[x][y].set_title(f"Bit width: {i+1}")
    if i == 0:
        axs[x][y].legend(["Floating", "Quantized: <1,1>"])
    else:
        axs[x][y].legend(["Floating", f"Quantized: <{i+1},1>", f"Quantized: <{i+1},2>"])

In [ ]:
# SOLUTION: you can base your solution using the skeleton code below
# We will keep the integer bit fixed at 1 (remaining bits are for fractional component)
bit_widths_qkeras = [1, 2, 3, 4, 5, 6]
N_epochs = 2

hls_accuracy_qkeras_list = []
hls_summary_qkeras_list = []

for bit_width in bit_widths_qkeras:
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print(f" --> QKeras model with bit width: {bit_width}")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

    # Build the QKeras model, setting the first argument in the quantized_bits function to bit_width
    # -- ADD CODE HERE --

    # Initiate the optimiser, compile the QKeras model, and initiate the weights according to the baseline model
    # -- ADD CODE HERE -- 

    # Train for small number of epochs (=2)
    # -- ADD CODE HERE --

    # Build the hls4ml config from the trained model, using granularity="name"
    # -- ADD CODE HERE --

    # Convert and compile the model
    # -- ADD CODE HERE --

    # Calculate the accuracy and store it
    # -- ADD CODE HERE --
    hls_accuracy_qkeras_list.append(accuracy)

    # Emulate the build and store the resource usage summary
    # -- ADD CODE HERE --
    with open("PATH-TO-REPORT/summary.ndjson", "r") as f:
        summary = ndjson.load(f)
    hls_summary_qkeras_list.append(summary)

In [ ]:
# SOLUTION: extract resource usage from summary jsons
resource_usage_qkeras_list = {}
for summ in hls_summary_qkeras_list:
    resource_names = list(filter(lambda x: x["name"] == "Estimated Resource Usage", summ))[0]['columns'][1:-1]
    available = list(filter(lambda x: x["name"] == "Available", summ))[0]['data'][:-1]
    estimated_resources = list(filter(lambda x: x["name"] == "Total", summ))[0]['data'][:-1]

    for i, resource in enumerate(resource_names):
        r = resource.split(" ")[0]
        if r not in resource_usage_qkeras_list:
            resource_usage_qkeras_list[r] = [float(estimated_resources[i])/float(available[i])]
        else:
            resource_usage_qkeras_list[r].append(float(estimated_resources[i])/float(available[i]))

In [ ]:
# SOLUTION: plot accuracy and resource-usage vs bit-width for baseline training and for QKeras training
fig, axs = plt.subplots(2,3, figsize=(16,8))

# -- ADD CODE HERE --

<div style="background-color:#FFCCCC">
    
### Questions (assessed)

* Please upload a screenshot of the graphs you have produced to show how the accuracy and resource-usage (LUTs, FFs, BRAM, DSPs, MLABs) vary as a function of the bit-widths for both the baseline and quantization-aware (QKeras) training [5 marks]
* Write a small summary paragraph (or a few bullet points) explaining the results you have obtained [3 marks]

## Part 5: Pruning

In this section we will discuss another method for reducing the resource usage, known as pruning (compression).

Weight matrices in neural networks can be huge. However not every weight contributes equally to the model performance. Pruning removes the low magnitude weights with very small impact to the classification performance.

On FPGAs, the weight matrices are known at compile time. Weights that are zero can be omitted by the compiler which of course reduces resource usage. 

To obtain sparse weight matrices, we can use L1 regularization to reduce weight magnitude. Then pruning can be scheduled to incrementaly remove and mask low magnitude weights.

Let's first import the relevant functionalities from keras.

In [ ]:
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning

Now we will set the pruning parameters (sparsity = 75%) and prune the baseline model (without QKeras training)

In [ ]:
pruning_params = {"pruning_schedule": pruning_schedule.ConstantSparsity(0.75, begin_step=100, frequency=100)}
pmodel = prune.prune_low_magnitude(model, **pruning_params)

Let's then retrain the pruned network

In [ ]:
adam = Adam(learning_rate=0.0001)
pmodel.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])

pmodel.fit(
    X_train_val,
    y_train_val,
    batch_size=128,
    epochs=5,
    validation_split=0.25,
    shuffle=True,
    callbacks=[pruning_callbacks.UpdatePruningStep()],
)

In [ ]:
# Strip pruning wrappers before converting for inference
pmodel = strip_pruning(pmodel)

In [ ]:
# Print percentage sparsity (i.e. percentage of zero weights) for each layer
for i, l in zip(pmodel.get_weights(), pmodel.layers):
    print(f"{np.sum(i.flatten()==0)/i.flatten().shape[0]*100:.1f}% sparsity for layer {l.name}")

Now we have trained a pruned neural network with 75% of weights stripped (zero), that achieves roughly the same accuracy as the baseline model. To highlight the impact of this, we will convert the model to FPGA firmware using hls4ml, and examine the resource usage.

In [ ]:
config = hls4ml.utils.config_from_keras_model(pmodel, granularity='model', backend='oneAPI')
print("-----------------------------------")
print("Configuration")
print(config)
print("-----------------------------------")
hls_model = hls4ml.converters.convert_from_keras_model(
    pmodel, hls_config=config, backend='oneAPI', output_dir='model_3/hls4ml_prj', part='Agilex7'
)

In [ ]:
hls_model.compile()

In [ ]:
# Extract the accuracy on the test dataset
X_test = np.ascontiguousarray(X_test)
y_hls = hls_model.predict(X_test)
hls_accuracy_prune = np.sum(np.argmax(y_test, axis=1) == np.argmax(y_hls, axis=1))/y_test.shape[0]
print(f"Achieved test accuracy of {(100 * hls_accuracy):.2f}%")

In [ ]:
# Emulate the build
hls_model.build(build_type='report')

In [ ]:
# Extract the resource usage from summary json and print
with open("model_3/hls4ml_prj/build/myproject.report.prj/reports/resources/json/summary.ndjson", "r") as f:
    summary = ndjson.load(f)

resource_names = list(filter(lambda x: x["name"] == "Estimated Resource Usage", summary))[0]['columns'][1:-1]
available = list(filter(lambda x: x["name"] == "Available", summary))[0]['data'][:-1]
estimated_resources = list(filter(lambda x: x["name"] == "Total", summary))[0]['data'][:-1]

print("~~~~~~~~~~~~~~ Resource usage (pruned model) ~~~~~~~~~~~~~~")
for i, resource in enumerate(resource_names):
    print(f"--> {resource}:")
    print(f"      * Available resource: {available[i]}")
    print(f"      * Used resource (estimated): {estimated_resources[i]}")
    print(f"      * Percentage of used resource (estimated): {100*float(estimated_resources[i])/float(available[i]):.2f}%")

<div style="background-color:#FFE5CC">

### Questions (unassessed)
* How does the accuracy and resource usage compare to the baseline model trained in Part 3?

<div style="background-color:#C2F5DD">


### Exercise

Train models with different sparsity. Plot the performance and resource usage as function of model sparsity.

We will investigate how large the savings in resource usage are in comparison to the baseline model before the accuracy drops.

</div>

In [ ]:
# SOLUTION: you can base your solution using the skeleton code below
# Loop over different sparsity values
# Note a sparsity of zero equals the baseline model (no pruning)
sparsities = [0., 0.5, 0.75, 0.8, 0.85, 0.9, 0.95, 0.99]
N_epochs = 5

hls_accuracy_prune_list = []
hls_summary_prune_list = []

for sparsity in sparsities:
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print(f" --> Pruning with sparsity: {sparsity}")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

    # Apply pruning to baseline model
    pruning_params = {"pruning_schedule": pruning_schedule.ConstantSparsity(sparsity, begin_step=100, frequency=100)}
    pmodel = prune.prune_low_magnitude(model, **pruning_params)

    # Initialise optimiser, compile and train model for 5 epochs
    # -- ADD CODE HERE --

    # Strip model
    pmodel = strip_pruning(pmodel)

    # Initialise hls4ml config and perform conversion to FPGA firmware
    # -- ADD CODE HERE --

    # Extract the accuracy on the test dataset
    # -- ADD CODE HERE --
    hls_accuracy_prune_list.append(accuracy)

    # Emulate the build
    # -- ADD CODE HERE --

    # Extract the resource usage and append the summary to list
    with open("PATH-TO-REPORT/summary.ndjson", "r") as f:
        summary = ndjson.load(f)
    hls_summary_prune_list.append(summary)

In [ ]:
# SOLUTION: extract resource usage from summary jsons
resource_usage_prune_list = {}
for summ in hls_summary_prune_list:
    resource_names = list(filter(lambda x: x["name"] == "Estimated Resource Usage", summ))[0]['columns'][1:-1]
    available = list(filter(lambda x: x["name"] == "Available", summ))[0]['data'][:-1]
    estimated_resources = list(filter(lambda x: x["name"] == "Total", summ))[0]['data'][:-1]

    for i, resource in enumerate(resource_names):
        r = resource.split(" ")[0]
        if r not in resource_usage_prune_list:
            resource_usage_prune_list[r] = [float(estimated_resources[i])/float(available[i])]
        else:
            resource_usage_prune_list[r].append(float(estimated_resources[i])/float(available[i]))

In [ ]:
# SOLUTION: plot accuracy and resource usage as function of sparsity
fig, axs = plt.subplots(2,3, figsize=(16,8))

# -- ADD CODE HERE --

<div style="background-color:#FFCCCC">
    
### Questions (assessed)

* Please upload a screenshot of the graphs you have produced to show how the accuracy and resource-usage (LUTs, FFs, BRAM, DSPs, MLABs) vary as a function of network sparsity? [5 marks]
* Write a small summary paragraph (or a few bullet points) explaining the results you have obtained. [3 marks]

<div style="background-color:#FFCCCC">

### Competition (extension)

Now you have two methods for reducing the resource usage, whilst maintaining a high classification accuracy:

* Quantization-aware training
* Pruning

The competition is to find the student who can **achieve the lowest resource usage for an accuracy on the test dataset of greater than 74%**.

The metric for resource usage is: (Fraction of used ALUTs) + (Fraction of used FFs) i.e.
`resource_usage['ALUTs']+resource_usage['ALUTs']`

**Tip**: you should train and synthesize a model that uses both quantization-aware training and pruning. Remember that the quantization is highly customizable (different numbers for each layer), and that different methods of compression exist.

Upload your best result via the same google forms. You will need to provide:
* Written summary of your approach (bullet points are fine)
* Print statement outputs of the global accuracy and resource-usage
* Code used to obtain solution (everything below this block in the notebook)
* Screenshots of any graphs you produce

The results of this competition will be used to differentiate between the top performing students. There are 5 additional marks available here.

In [ ]:
# -- ADD CODE HERE --